In [1]:
import pandas as pd
import re
import xarray as xr
import rasterio as rio
import rioxarray
import os
from glob import glob
from shapely.geometry import box
import numpy as np

In [2]:
def snodas_for_aso(aso_raster_fn):
    # open ASO raster and extract timestamp and bounds
    aso_raster = rioxarray.open_rasterio(aso_raster_fn).squeeze()
    time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))

    # download and unzip snodas snow depths
    snodas_url = f"https://noaadata.apps.nsidc.org/NOAA/G02158/masked/{time.year}/{time.strftime("%m")}_{time.strftime("%b")}/SNODAS_{time.strftime("%Y%m%d")}.tar"
    os.makedirs('../../data/tmp/snodas', exist_ok=True)
    !wget -P ../../data/tmp/snodas -nd --no-check-certificate --reject "index.html*" -np -e robots=off $snodas_url
    snodas_fn = f"../../data/tmp/snodas/SNODAS_{time.strftime("%Y%m%d")}.tar"
    !tar -xf $snodas_fn -C ../../data/tmp/snodas
    snodas_depth_fn = "../../data/tmp/snodas/us_ssmv11036*.gz"
    !gunzip $snodas_depth_fn
    
    # crop, reproject, and mask snodas snow depths
    snodas_da = rioxarray.open_rasterio(glob('../../data/tmp/snodas/us_ssmv11036*.txt')[0]).squeeze()
    snodas_clipped_da = snodas_da.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    snodas_resampled_da = snodas_clipped_da.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)
    snodas_resampled_da = snodas_resampled_da.where(snodas_resampled_da != -9999)/1000

    # save to netcdf and remove temporary data
    snodas_resampled_da.to_netcdf(f'../../data/snodas/snodas_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')
    !rm ../../data/tmp/snodas/*

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_4172/1678106873.py:4: SyntaxWarning: invalid escape sequence '\d'
  time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())


In [3]:
aso_raster_fns = glob('../../data/ASO/ASO_50m_SD_withS1overpass/*/ASO*.tif')

In [4]:
for i, aso_raster_fn in enumerate(aso_raster_fns):
    error_list = []
    print(f'----\nworking on {aso_raster_fn.split("/")[-1]}, {i+1}/{len(aso_raster_fns)}\n----')
    try: 
        snodas_for_aso(aso_raster_fn)
    except:
        print('error, skipping')
        error_list.append(aso_raster_fn)

----
working on ASO_50M_SD_American_20230131_clean.tif, 1/233
----
--2025-04-14 11:07:27--  https://noaadata.apps.nsidc.org/NOAA/G02158/masked/2023/01_Jan/SNODAS_20230131.tar
Resolving noaadata.apps.nsidc.org (noaadata.apps.nsidc.org)... 128.138.135.43
Connecting to noaadata.apps.nsidc.org (noaadata.apps.nsidc.org)|128.138.135.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26101760 (25M) [application/octet-stream]
Saving to: ‘../../data/tmp/snodas/SNODAS_20230131.tar’

SNODAS_20230131.tar 100%[===================>]  24.89M  15.2MB/s    in 1.6s    

2025-04-14 11:07:29 (15.2 MB/s) - ‘../../data/tmp/snodas/SNODAS_20230131.tar’ saved [26101760/26101760]

----
working on ASO_50M_SD_American_20230413_clean.tif, 2/233
----
--2025-04-14 11:07:31--  https://noaadata.apps.nsidc.org/NOAA/G02158/masked/2023/04_Apr/SNODAS_20230413.tar
Resolving noaadata.apps.nsidc.org (noaadata.apps.nsidc.org)... 128.138.135.43
Connecting to noaadata.apps.nsidc.org (noaadata.apps.nsid

In [5]:
error_list

[]